In [ ]:
# Generating coordinates/latitude/longitude
import rasterio
import numpy as np
import pandas as pd
from pyproj import Transformer

# Assuming file_path is defined and points to your GeoTIFF file
file_path = "C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\Landsat8\\2022\\L8_UTC_20221111_031713.tif"

# Open your GeoTIFF file
with rasterio.open(file_path) as src:
    array = src.read()
    transform = src.transform
    src_crs = src.crs  # Source CRS
    dest_crs = 'EPSG:4326'  # WGS 84
    
    # Get arrays of column and row indices
    cols, rows = np.meshgrid(np.arange(array.shape[2]), np.arange(array.shape[1]))

    # Convert meshgrid arrays to coordinate arrays using rasterio's method, which are 2D
    xs, ys = rasterio.transform.xy(transform, rows, cols, offset='center')

    # Flatten the coordinate arrays to pass to transform function
    lon, lat = transformer.transform(np.array(xs).flatten(), np.array(ys).flatten())

    # Flatten the arrays if necessary and create a DataFrame
    df = pd.DataFrame({
        'Longitude': lon,
        'Latitude': lat
    })

    # Add pixel values from each band to the DataFrame
    band_names = src.descriptions
    for i, band_name in enumerate(band_names):
        df[band_name] = array[i].flatten()

    df['ST_B10_Celsius'] = df['ST_B10'] * 0.00341802 + 149 - 273.15
    
    # Scaling and offsetting specific bands
    bands_to_scale = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
    for band in bands_to_scale:
        df[f"{band}_Scaled"] = df[band] * 2.75e-05 - 0.2

    additional_scales = {
        'ST_ATRAN': 0.0001, 'ST_CDIST': 0.01, 'ST_DRAD': 0.001, 
        'ST_EMIS': 0.0001, 'ST_EMSD': 0.0001, 'ST_QA': 0.01, 
        'ST_TRAD': 0.001, 'ST_URAD': 0.001
    }
    for band, scale in additional_scales.items():
        df[f"{band}_Scaled"] = df[band] * scale

    # Optionally, handle NoData values
    nodata_values = src.nodatavals
    for i, nodata_value in enumerate(nodata_values):
        if nodata_value is not None:
            df[band_name].replace(nodata_value, np.nan, inplace=True)

print(df[['Latitude', 'Longitude']].head())

In [ ]:
# Plotting heatmap
import folium
from folium.plugins import HeatMap
from branca.colormap import LinearColormap
import pandas as pd

# First, clean your data by dropping or filling NaN values
df = df.dropna(subset=['ST_B10_Celsius'])  # Removes rows where 'ST_B10_Celsius' is NaN

# Create a map centered around the mean coordinates of the filtered data
map_center = [df['Latitude'].mean(), df['Longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=11)

# Create the heatmap
heat_data = [[row['Latitude'], row['Longitude'], row['ST_B10_Celsius']] for index, row in df.iterrows()]
HeatMap(
    heat_data,
    min_opacity=0.5,
    radius=10,  # Adjust radius to control the spread of the heatmap
    blur=14,    # Adjust blur for visual smoothness
    gradient={0.2: 'blue', 0.4: 'cyan', 0.6: 'lime', 0.8: 'yellow', 1: 'red'}  # Define color gradient
).add_to(m)

# Create a color scale legend
color_scale = LinearColormap(
    ['blue', 'cyan', 'lime', 'yellow', 'red'],
    vmin=df['ST_B10_Celsius'].min(),
    vmax=df['ST_B10_Celsius'].max()
)
color_scale.caption = 'Land Surface Temperature (Celsius)'
m.add_child(color_scale)

# Save or display the map
m.save('C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\MSE-ES-UHI\\L8_UTC_20221111_031713_Heatmap.html')  # Save the map to HTML file
m  # Display the map in a Jupyter notebook if you are using one